In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

from statsforecast import StatsForecast
from statsforecast.models import ARIMA, AutoARIMA

from utilsforecast.losses import *
from utilsforecast.evaluation import evaluate

warnings.filterwarnings("ignore")
os.environ["NIXTLA_ID_AS_COL"] = "true"
pd.set_option('display.precision', 3)

In [ ]:
plt.rcParams['figure.figsize'] = (9,6)

In [ ]:
url = "https://raw.githubusercontent.com/marcopeix/TimeSeriesForecastingUsingFoundationModels/refs/heads/main/data/walmart_sales_small.csv"

df = pd.read_csv(url, parse_dates=["Date"])
df = df[['Store', 'Date', 'Weekly_Sales']]
df.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    plot_df = df[df['Store'] == i+1]

    ax.plot(plot_df['Date'], plot_df['Weekly_Sales'])
    ax.set_title(f'Store {i+1}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Sales ($)')

fig.autofmt_xdate()
plt.tight_layout()

# Moving average - MA(q)

In [ ]:
fig, ax = plt.subplots()

ax.plot(ma1_process)
ax.set_xlabel('Timesteps')
ax.set_ylabel('Value')

plt.title('Simulated MA(1) process')

plt.tight_layout()

## Forecasting with MA(q)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    plot_df = df[df['Store'] == i+1]
    cv_plot_df = cv_df[cv_df['Store'] == i+1]

    ax.plot(plot_df['Date'], plot_df['Weekly_Sales'])
    ax.plot(cv_plot_df['Date'], cv_plot_df['MA'], label='MA(q)')
    ax.fill_between(cv_plot_df['Date'], cv_plot_df['MA-lo-80'], cv_plot_df['MA-hi-80'], color='green', alpha=0.2)
    ax.legend()
    ax.set_title(f'Store {i+1}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Sales ($)')

fig.autofmt_xdate()
plt.tight_layout()

# Autoregressive model - AR(p)

In [ ]:
plot_acf(AR2_process, lags=20);

plt.ylim(-0.5, 1.5)
plt.tight_layout()

## Forecasting with AR(p)

In [ ]:
# Use an AR model


sf = StatsForecast(models=[ma_model, ar_model], freq='W')
cv_df = sf.cross_validation(h=horizon, 
                            df=df, 
                            n_windows=10, 
                            step_size=horizon, 
                            level=[80], 
                            time_col='Date', 
                            id_col='Store', 
                            target_col='Weekly_Sales')

cv_df.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    plot_df = df[df['Store'] == i+1]
    cv_plot_df = cv_df[cv_df['Store'] == i+1]

    ax.plot(plot_df['Date'], plot_df['Weekly_Sales'])
    ax.plot(cv_plot_df['Date'], cv_plot_df['AR'], label='AR(p)')
    ax.fill_between(cv_plot_df['Date'], cv_plot_df['AR-lo-80'], cv_plot_df['AR-hi-80'], color='green', alpha=0.2)
    ax.legend()
    ax.set_title(f'Store {i+1}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Sales ($)')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
eval_df = cv_df.drop(['Date', 'cutoff'], axis=1)
evaluation = evaluate(df=eval_df, metrics=[mae, smape], target_col='Weekly_Sales', id_col='Store')
avg_evaluation = evaluation.drop(['Store'], axis=1).groupby('metric').mean().reset_index()
avg_evaluation

# Forecasting with ARIMA (AutoARIMA)

In [ ]:
# Use AutoARIMA



sf = StatsForecast(models=[ma_model, ar_model, auto_arima_model], freq='W')
cv_df = sf.cross_validation(h=horizon, 
                            df=df, 
                            n_windows=10, 
                            step_size=horizon, 
                            level=[80], 
                            time_col='Date', 
                            id_col='Store', 
                            target_col='Weekly_Sales')

cv_df.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12,9))

for i, ax in enumerate(axes.flatten()):
    plot_df = df[df['Store'] == i+1]
    cv_plot_df = cv_df[cv_df['Store'] == i+1]

    ax.plot(plot_df['Date'], plot_df['Weekly_Sales'])
    ax.plot(cv_plot_df['Date'], cv_plot_df['AutoARIMA'], label='ARIMA(p,d,q)')
    ax.fill_between(cv_plot_df['Date'], cv_plot_df['AutoARIMA-lo-80'], cv_plot_df['AutoARIMA-hi-80'], color='green', alpha=0.2)
    ax.legend()
    ax.set_title(f'Store {i+1}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Weekly Sales ($)')

fig.autofmt_xdate()
plt.tight_layout()

In [ ]:
eval_df = cv_df.drop(['Date', 'cutoff'], axis=1)
evaluation = evaluate(df=eval_df, metrics=[mae, smape], target_col='Weekly_Sales', id_col='Store')
avg_evaluation = evaluation.drop(['Store'], axis=1).groupby('metric').mean().reset_index()
avg_evaluation